<font color=red>注：此处是文档第22页</font>

## PyTorch自动微分

autograd 包是 PyTorch 中所有神经网络的核心。首先让我们简要地介绍它，然后我们将会去训练我们的第一个神经网络。该 autograd 软件包为 Tensors 上的所有操作提供自动微分。它是一个由运行定义的框架，这意味着以代码运行方式定义你的后向传播，并且每次迭代都可以不同。我们从 tensor 和 gradients 来举一些例子。

**1、TENSOR**

torch.Tensor是包的核心类。如果将其属性 **.requires_grad** 设置为True，则会开始跟踪针对tensor的所有操作。完成计算后，您可以调用 **.backward()** 来自动计算所有梯度。该张量的梯度将累积到 **.grad** 属性中。

要停止 tensor 历史记录的跟踪，您可以调用 **.detach()**，它将其与计算历史记录分离，并防止将来的计算被跟踪。

要停止跟踪历史记录（和使用内存），您还可以将代码块使用 **with torch.no_grad():** 包装起来。在评估模型时，这是特别有用，因为模型在训练阶段具有requires_grad = True 的可训练参数有利于调参，但在评估阶段我们不需要梯度。

还有一个类对于 autograd 实现非常重要那就是 **Function**。Tensor 和 Function 互相连接并构建一个非循环图，它保存整个完整的计算过程的历史信息。每个张量都有一个 **.grad_fn** 属性保存着创建了张量的 Function 的引用，（如果用户自己创建张量，则grad_fn 是 None）。

如果你想计算导数，你可以调用 **Tensor.backward()**。如果 Tensor 是标量（即它包含一个元素数据），则不需要指定任何参数backward()，但是如果它有更多元素，则需要指定一个 **gradient** 参数来指定张量的形状。


In [19]:
import torch

In [20]:
x1 = torch.ones(2, 2, requires_grad=True)
x2 = torch.ones(2, 2, requires_grad=False)
print(x1)
print(x2)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[1., 1.],
        [1., 1.]])


In [21]:
y1 = x1 + 2
y2 = x2 + 2
print(y1)
print(y2)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[3., 3.],
        [3., 3.]])


In [22]:
print(y1.grad_fn)
print(y2.grad_fn)

None


In [23]:
z1 = y1 * y1 * 3
z2 = y2 * y2 * 3

out1 = z1.mean()
out2 = z2.mean()

print(z1)
print(z2)
print(out1)
print(out2)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor([[27., 27.],
        [27., 27.]])
tensor(27., grad_fn=<MeanBackward0>)
tensor(27.)


.requires_grad_( … ) 会改变张量的 requires_grad 标记。输入的标记默认为 False ，如果没有提供
相应的参数。

In [24]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


梯度：

我们现在后向传播，因为输出包含了一个标量，out.backward() 等同于

out.backward(torch.tensor(1.))。

In [25]:
out1.backward()

打印梯度 d(out)/dx

In [26]:
print(x1.grad)
print(x2.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
None


x1 = torch.ones(2, 2, requires_grad=True)  
y1 = x1 + 2  
z1 = y1 * y1 * 3  
out1 = z1.mean()  
此时对应公式：$$o = \frac{1}{4}\sum_{i}z_i$$
$$z_i=(3x + 2)^2$$

一个雅可比向量积的例子：

In [27]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    print(y)
    y = y * 2
print(y)

tensor([-0.1021,  1.4288,  2.3268], grad_fn=<MulBackward0>)
tensor([-0.2042,  2.8577,  4.6535], grad_fn=<MulBackward0>)
tensor([-0.4083,  5.7154,  9.3071], grad_fn=<MulBackward0>)
tensor([-0.8167, 11.4308, 18.6141], grad_fn=<MulBackward0>)
tensor([-1.6333, 22.8615, 37.2282], grad_fn=<MulBackward0>)
tensor([-3.2667, 45.7230, 74.4564], grad_fn=<MulBackward0>)
tensor([ -6.5333,  91.4460, 148.9129], grad_fn=<MulBackward0>)
tensor([-13.0667, 182.8921, 297.8257], grad_fn=<MulBackward0>)
tensor([-26.1333, 365.7841, 595.6515], grad_fn=<MulBackward0>)
tensor([ -52.2666,  731.5682, 1191.3030], grad_fn=<MulBackward0>)


In [28]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


你可以通过将代码包裹在 with torch.no_grad()，来停止对从跟踪历史中 的 .requires_grad=True 的
张量自动求导。

In [29]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
